# PREDICTION DES RETARDS DE VOL


## 1. IMPORT DES LIBRAIRIES


In [2]:
import pandas as pd
from os.path import join

## 2. CHARGEMENT DES DONNÉES


Les données sont réparties en 12 fichiers csv, un par mois de l'année 2016.  
Chaque fichier comporte plus de 470000 lignes. Cela représente un total de plus de 5,5 millions de lignes. Il nous faut donc trouver une stratégie pour épargner notre mémoire.


### 2.1. Exploration d'un unique (`2016_01.csv`)

Dans ce cadre, nous décidons de commencer par analyser un fichier pour identifier les optimisations possibles afin de les appliquer à tous les autres fichiers.


In [3]:
raw_data_path = join("..", "data", "raw")

jan_data_path = join(raw_data_path, "2016_01.csv")

In [4]:
df_jan = pd.read_csv(jan_data_path, on_bad_lines="warn", low_memory=False)
df_jan.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,TAIL_NUM,...,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,Unnamed: 64
0,2016,1,1,6,3,2016-01-06,AA,19805,AA,N4YBAA,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,1,1,7,4,2016-01-07,AA,19805,AA,N434AA,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,1,1,8,5,2016-01-08,AA,19805,AA,N541AA,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,1,1,9,6,2016-01-09,AA,19805,AA,N489AA,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,1,1,10,7,2016-01-10,AA,19805,AA,N439AA,...,4,0.0,0.0,47.0,0.0,66.0,NaN,NaN,NaN,NaN


In [5]:
columns = df_jan.columns
columns

Index(['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
       'UNIQUE_CARRIER', 'AIRLINE_ID', 'CARRIER', 'TAIL_NUM', 'FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID',
       'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_FIPS',
       'ORIGIN_STATE_NM', 'ORIGIN_WAC', 'DEST_AIRPORT_ID',
       'DEST_AIRPORT_SEQ_ID', 'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME',
       'DEST_STATE_ABR', 'DEST_STATE_FIPS', 'DEST_STATE_NM', 'DEST_WAC',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'DEP_DELAY_NEW', 'DEP_DEL15',
       'DEP_DELAY_GROUP', 'DEP_TIME_BLK', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'ARR_DELAY_NEW', 'ARR_DEL15', 'ARR_DELAY_GROUP', 'ARR_TIME_BLK',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'FLIGHTS', 'DISTANCE',
       'DISTANCE_GROUP', 'CARRIER_DELAY', 'WEATHER_DELAY

#### 2.1.1 Détermination de la cible

Nous avons identifié plusieurs candidats pour la cible : ARR_DELAY, ARR_DELAY_NEW, ARR_DEL15 et ARR_DELAY_GROUP


In [6]:
potential_targets = ["ARR_DELAY", "ARR_DELAY_NEW", "ARR_DEL15", "ARR_DELAY_GROUP"]

for target in potential_targets:
    print(f"{target}: Nombre de Valeurs Uniques")
    print("-" * 50)
    unique_count = df_jan[target].dropna().nunique()
    print(unique_count)
    if unique_count == 2:
        print(df_jan[target].dropna().unique())
    print("-" * 50)

ARR_DELAY: Nombre de Valeurs Uniques
--------------------------------------------------
735
--------------------------------------------------
ARR_DELAY_NEW: Nombre de Valeurs Uniques
--------------------------------------------------
660
--------------------------------------------------
ARR_DEL15: Nombre de Valeurs Uniques
--------------------------------------------------
2
[0. 1.]
--------------------------------------------------
ARR_DELAY_GROUP: Nombre de Valeurs Uniques
--------------------------------------------------
15
--------------------------------------------------


Nous constatons que seuls la colonne `ARR_DEL15` est sous forme binaire (en retard ou à l'heure) conformément à ce que nous souhaitons prédire. Les autres colonnes sont redondantes et peuvent donc être supprimées.


In [7]:
target = "ARR_DEL15"
redundant_targets = [col for col in potential_targets if col is not target]

print(redundant_targets)

['ARR_DELAY', 'ARR_DELAY_NEW', 'ARR_DELAY_GROUP']


Analysons cette colonne cible :


In [8]:
df_jan[target].info()

<class 'pandas.core.series.Series'>
RangeIndex: 445827 entries, 0 to 445826
Series name: ARR_DEL15
Non-Null Count   Dtype  
--------------   -----  
433298 non-null  float64
dtypes: float64(1)
memory usage: 3.4 MB


Nous constatons que le type de la colonne (float) n'est pas adapté. Il faudrait le transformer en bouléen.


In [9]:
target_na_count = df_jan[target].isna().sum()
print(f"{target_na_count} lignes vide pour la cible {target}")

12529 lignes vide pour la cible ARR_DEL15


In [10]:
for col in redundant_targets:
    na_count = df_jan[col].isna().sum()
    print(f"{na_count} lignes vide pour la colonne {col}")

12529 lignes vide pour la colonne ARR_DELAY
12529 lignes vide pour la colonne ARR_DELAY_NEW
12529 lignes vide pour la colonne ARR_DELAY_GROUP


In [11]:
all_na_mask = df_jan[potential_targets].isna().all(axis=1)
all_na_count = all_na_mask.sum()

print(f"Lignes où TOUTES les colonnes sont NA : {all_na_count}")

Lignes où TOUTES les colonnes sont NA : 12529


Nous constatons que plusieurs milliers de lignes n'ont pas de valeur cible. L'exploration des potentielles colonnes cibles précédemment écartées nous indique qu'il ne serait pas possible de reconstituer les valeurs manquantes avec ces autres colonnes.  
Il semble donc indispensable de les supprimer car ces lignes ne peuvent pas permettre d'entraîner le futur modèle.


In [12]:
df_jan = df_jan.dropna(subset=["ARR_DEL15"])

In [33]:
jan_delay_rate = df_jan["ARR_DEL15"].mean()
print(f"Le taux de retard à l'arrivée en janvier est de {jan_delay_rate:.2f}")

Le taux de retard à l'arrivée en janvier est de 0.16


#### 2.1.2. Identifications des colonnes aux valeurs manquantes


In [13]:
na_percentages = (df_jan.isna().sum() / len(df_jan)) * 100

high_na_cols = na_percentages[na_percentages > 80].index.tolist()

print("Colonnes avec plus de 50% de valeurs manquantes :")
for col in high_na_cols:
    print(f"- {col}: {na_percentages[col]:.2f}%")

print(f"\nListe des colonnes ({len(high_na_cols)}) :")
print(high_na_cols)

Colonnes avec plus de 50% de valeurs manquantes :
- CANCELLATION_CODE: 100.00%
- CARRIER_DELAY: 83.64%
- WEATHER_DELAY: 83.64%
- NAS_DELAY: 83.64%
- SECURITY_DELAY: 83.64%
- LATE_AIRCRAFT_DELAY: 83.64%
- FIRST_DEP_TIME: 99.44%
- TOTAL_ADD_GTIME: 99.44%
- LONGEST_ADD_GTIME: 99.44%
- Unnamed: 64: 100.00%

Liste des colonnes (10) :
['CANCELLATION_CODE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'FIRST_DEP_TIME', 'TOTAL_ADD_GTIME', 'LONGEST_ADD_GTIME', 'Unnamed: 64']


Nous avons identifié les colonnes pour lesquelles plus de 80% des données sont manquantes. Sans analyse plus approfondie, nous pourrions nous dire que ces données sont inutilisables, car avec un tel taux de données manquantes, il serait hasardeux d'essayer d'imputer les valeurs manquantes sur aussi peu de données. Attardons nous cependant sur ces colonnes.

**Analyse colonne par colonne :**

- `Unnamed: 64`: 100.00% => colonne entièrement vide, probablement due à une virgule surnuméraire en fin de ligne. Colonne à supprimer définitivement.
- `CANCELLATION_CODE`: 97.38% => ici, nous supposons que les valeurs vides correspondent tout simplement à une absence de code d'annulation dans le cas où les vols ne sont pas annulés
  Les colonnes suivantes ne sont pas décrites dans le sujet d'examen. Nous pouvons cependant réaliser quelques suppositions :
- `CARRIER_DELAY`, `WEATHER_DELAY`, `NAS_DELAY`, `SECURITY_DELAY` et `LATE_AIRCRAFT_DELAY`: toutes à 84.10% => Nous supposons que les valeurs nulles correspondent aux vols non retardés
- `FIRST_DEP_TIME`, `TOTAL_ADD_GTIME` et `LONGEST_ADD_GTIME`: toutes à 99.43% => Nous supposons que les valeurs manquantes correspondent aux vols n'ont pas été détournés.

Nous allons analyser ces hypothèses.


In [14]:
mask_na = df_jan["CANCELLATION_CODE"].isna()
mask_is_not_cancelled = df_jan["CANCELLED"] == 0.0

if (mask_na == mask_is_not_cancelled).all():
    print(
        "Les valeurs NA de la colonne CANCELLATION_CODE correspondent aux vols non annulés."
    )
else:
    print(
        "Les valeurs NA de la colonne CANCELLATION_CODE ne correspondent pas aux vols non annulés."
    )
    print(df_jan[mask_na != mask_is_not_cancelled])


Les valeurs NA de la colonne CANCELLATION_CODE correspondent aux vols non annulés.


L'hypothèse selon laquelle Les valeurs NA de la colonne CANCELLATION_CODE correspondent aux vols non annulés est vérifiée. Il serait envisageable d'imputer les valeurs manquantes avec un 4e code d'annulation, correspondant virtuellement à une non-annulation.


In [15]:
mask_all_na = (
    df_jan[
        [
            "CARRIER_DELAY",
            "WEATHER_DELAY",
            "NAS_DELAY",
            "SECURITY_DELAY",
            "LATE_AIRCRAFT_DELAY",
        ]
    ]
    .isna()
    .all(axis=1)
)

print(
    f"Nombre de lignes où toutes les colonnes de type de délai sont NA : {mask_all_na.sum()}"
)

mask_is_not_delayed = df_jan["ARR_DEL15"] == 0.0

if (mask_all_na == mask_is_not_delayed).all():
    print(
        "\nLes lignes où toutes les colonnes de type de délai sont NA correspondent aux vols non retardés."
    )
else:
    print("\nIl existe des exceptions :")
    print(
        df_jan[mask_all_na != mask_is_not_delayed][
            [
                "ARR_DELAY",
                "CARRIER_DELAY",
                "WEATHER_DELAY",
                "NAS_DELAY",
                "SECURITY_DELAY",
                "LATE_AIRCRAFT_DELAY",
                "ARR_DEL15",
            ]
        ]
    )

Nombre de lignes où toutes les colonnes de type de délai sont NA : 362416

Les lignes où toutes les colonnes de type de délai sont NA correspondent aux vols non retardés.


A première vue, l'hypothèse selon laquelle les lignes où toutes les colonnes de type de délai sont NA correspondent aux vols non retardés, semble fausse. Cependant, les exceptions relevées semblent correspondre à des lignes où le statut en retard ou à l'heure est également indéterminé. Les Lignes où les deux masques correspondent pourraient être remplies avec des valeurs nulles (0min de retard pour cette cause).


In [16]:
mask_all_na = (
    df_jan[["FIRST_DEP_TIME", "TOTAL_ADD_GTIME", "LONGEST_ADD_GTIME"]]
    .isna()
    .all(axis=1)
)

print(
    f"Nombre de lignes où toutes les colonnes de type de time sont NA : {mask_all_na.sum()}"
)

mask_is_not_diverted = df_jan["DIVERTED"] == 0.0

if (mask_all_na == mask_is_not_diverted).all():
    print(
        "\nLes lignes où toutes les colonnes de type de délai sont NA correspondent aux vols non déviés."
    )
else:
    print("\nIl existe des exceptions :")
    print(
        df_jan[mask_all_na != mask_is_not_diverted][
            [
                "FIRST_DEP_TIME",
                "TOTAL_ADD_GTIME",
                "LONGEST_ADD_GTIME",
                "DIVERTED",
            ]
        ]
    )

Nombre de lignes où toutes les colonnes de type de time sont NA : 430889

Il existe des exceptions :
        FIRST_DEP_TIME  TOTAL_ADD_GTIME  LONGEST_ADD_GTIME  DIVERTED
70               642.0             11.0               11.0       0.0
305             1336.0             19.0               19.0       0.0
339             1053.0             12.0               12.0       0.0
397             1511.0             22.0               22.0       0.0
432              859.0             11.0               11.0       0.0
...                ...              ...                ...       ...
444013           751.0             14.0               14.0       0.0
444687          1106.0             12.0               12.0       0.0
445245          1213.0             30.0               30.0       0.0
445302           903.0             16.0               16.0       0.0
445543           733.0             57.0               57.0       0.0

[2409 rows x 4 columns]


Ici, l'hypothèse selon laquelle les lignes où toutes les colonnes de type de délai sont NA correspondent aux vols non déviés, semble fausse. Il semble donc hasardeux d'essayer d'imputer les valeurs manquantes.


#### 2.1.3. Identification des colonnes à risque de data leakage


Pour la suite de l'analyse, nous partirons du principe que le modèle s'attachera à prédire le retard d'un vol qui aura déjà décollé.

Parmis les colonnes restantes certaines contiennent des informations qui influencent directement le résultat et qui ne seront normalement pas présentes au moment de la prédiction :

- `ARR_TIME` : Heure réelle d'arrivée inconnues au moment de la prédiction
- `ARR_DELAY`, `ARR_DELAY_NEW`, `ARR_DEL15`, `ARR_DELAY_GROUP` : Variables cibles (à prédire)
- `WHEELS_ON`, `TAXI_IN` : Données post-départ
- `ACTUAL_ELAPSED_TIME`, `AIR_TIME` : Durées réelles inconnues a priori
- `CANCELLED` : Les vols annulés ne peuvent être à l'heure ou en retard
- `DIVERTED` : Statut du vol potentiellement inconnu au moment de la prédiction

Ces colonnes doivent donc être supprimées.


In [17]:
data_leakage_cols = [
    "ARR_TIME",
    "ARR_DELAY",
    "ARR_DELAY_NEW",
    "ARR_DELAY_GROUP",
    "WHEELS_ON",
    "TAXI_IN",
    "ACTUAL_ELAPSED_TIME",
    "AIR_TIME",
    "CANCELLED",
    "DIVERTED",
]

#### 2.1.4. Identification des colonnes redondantes


Parmi les colonnes restantes, nous identifions plusieurs colonnes redondantes entre elles :

- `ARR_TIME_BLK` et `ARR_TIME` sont redondantes. Nous conserverons `ARR_TIME` car il s'agit d'une valeur numérique continue plutôt que catégorielle, cela permettra plus de précision.
- `DEP_DELAY`, `DEP_DELAY_NEW`, `DEP_DEL15`, `DEP_DELAY_GROUP` et `DEP_TIME_BLK` représentent toutes les retard au départ. Nous conserverons `DEP_DELAY` car elle nous semble la plus précise de toutes ces valeurs : il s'agit d'une valeur numérique continue qui prend aussi en compte l'avance au départ (valeurs négatives) qui peut avoir une influence directe sur le retard à l'arrivée.
- `AIRLINE_ID`, `UNIQUE_CARRIER` et `CARRIER` représentent toutes trois une façon d'identifier les compagnies aériennes. Nous ne conserverons que `UNIQUE_CARRIER`.
- Parmis les identifiants multiples pour l'origine et la destination, les codes IATA présents dans `ORIGIN` et `DEST` sont suffisament précis pour identifier à eux seuls l'aéroport d'origine et celui de destination.
- `ORIGIN_STATE_NM` et `ORIGIN_STATE_ABR` sont clairement redondants, l'abréviation seule pourrait suffire. Cependant, toute l'information géographique peut être portée par `ORIGIN`.
- De la même façon, `DEST_STATE_NM` et `DEST_STATE_ABR` sont redondants. Même analyse que pour `ORIGIN`, ici nous ne conserverons que `DEST`.
- `DISTANCE` et `DISTANCE_GROUP` sont redondantes. Nous choisissons de conserver `DISTANCE` car il s'agit d'une valeur continue, plus facile à interpréter.


Les colonnes redondantes n'apportant pas plus d'information, pourront être supprimées.


In [18]:
redundant_cols = [
    # Arrival time related -> keeping ARR_TIME
    "ARR_TIME_BLK",
    # Departure related -> keeping DEP_DELAY
    "DEP_DELAY_NEW",
    "DEP_DEL15",
    "DEP_DELAY_GROUP",
    "DEP_TIME_BLK",
    # Airline/Carrier related -> keeping UNIQUE_CARRIER
    "AIRLINE_ID",
    "CARRIER",
    # Origin related -> keeping ORIGIN
    "ORIGIN_AIRPORT_ID",
    "ORIGIN_AIRPORT_SEQ_ID",
    "ORIGIN_CITY_MARKET_ID",
    "ORIGIN_STATE_FIPS",
    "ORIGIN_STATE_NM",
    "ORIGIN_WAC",
    "ORIGIN_CITY_NAME",
    "ORIGIN_STATE_ABR",
    # Destination related -> keeping DEST
    "DEST_AIRPORT_ID",
    "DEST_AIRPORT_SEQ_ID",
    "DEST_CITY_MARKET_ID",
    "DEST_STATE_FIPS",
    "DEST_STATE_NM",
    "DEST_WAC",
    "DEST_CITY_NAME",
    "DEST_STATE_ABR",
    # Distance related -> keeping DISTANCE
    "DISTANCE_GROUP",
]

#### 2.1.5. Analyse des colonnes temporelles


Parmi les colonnes temporelles (`YEAR`, `QUARTER`, `MONTH`, `DAY_OF_MONTH`,`DAY_OF_WEEK` et `FL_DATE`), il existe une certaine redondance.  
`FL_DATE` comporte toute l'information temporelle nécessaire en une seule colonne. Cependant, du point de vue du machine learning, il est plus facile de manipuler des valeurs numérique plutôt que des dates. Plusieurs stratégies s'offrent à nous :

- Tout conserver, mais cela ne semble pas très efficient,
- Ne conserver que `FL_DATE` dans notre future base de données et extraire les données temporelle en prétraitement
- Ne conserver que les données segmentées dans la base de données afin de les fournir directement lors de l'entrainement du modèle.


Nous choisirons la dernière méthode pour faciliter l'entraînement du modèle. Nous prenons églament le parti de supprimer la colonne `YEAR` car toutes nos données ne concernent que l'année 2016, cette colonne n'apporte donc aucune plus value à l'entraînement du modèle.


In [19]:
removable_temporal_cols = ["FL_DATE", "YEAR", "QUARTER"]

#### 2.1.6 Cas de la colonne `FLIGHTS`


In [20]:
df_jan["FLIGHTS"].value_counts()

FLIGHTS
1.0    433298
Name: count, dtype: int64

Nous constatons que cette colonne, qui n'est d'ailleurs pas décrite, ne comprend qu'une valeur unique. Elle n'est donc pas nécessaire à la suite de l'analyse.


In [21]:
useless_cols = ["FLIGHTS"]

#### 2.1.7. Cas de `FL_NUM` et de `TAIL_NUM`


In [22]:
fl_num_unique_count = df_jan["FL_NUM"].nunique()
tail_num_unique_count = df_jan["TAIL_NUM"].nunique()
print(f"nombre de valeurs uniques de FL_NUM: {fl_num_unique_count}")
print(f"nombre de valeurs uniques de TAIL_NUM: {tail_num_unique_count}")

nombre de valeurs uniques de FL_NUM: 6654
nombre de valeurs uniques de TAIL_NUM: 4236


Les numéros de vol et identifiants des appareils pourraient être pertinents dans notre prédiction. Cependant du fait du très grand nombre de catégorie et au vue de nos ressources limitées, nous les excluerons des colonnes à conserver pour notre modèle.


In [23]:
useless_cols += ["FL_NUM", "TAIL_NUM"]

### 2.2. Identification des colonnes à conserver

Maintenant que nous avons identifié toutes les colonnes à supprimer, nous allons pouvoir appliquer ce traitement à tous les fichiers fournis afin de les alléger et de pouvoir les traiter dans leur ensemble.


In [24]:
cols_to_drop = (
    redundant_targets
    + high_na_cols
    + data_leakage_cols
    + redundant_cols
    + removable_temporal_cols
    + useless_cols
)
cols_to_keep = [col for col in columns if col not in cols_to_drop]
target_col = "ARR_DEL15"

cols_to_keep

['MONTH',
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
 'UNIQUE_CARRIER',
 'ORIGIN',
 'DEST',
 'CRS_DEP_TIME',
 'DEP_TIME',
 'DEP_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'CRS_ARR_TIME',
 'ARR_DEL15',
 'CRS_ELAPSED_TIME',
 'DISTANCE']

### 2.3. Conversion des fichiers `.csv` en `.parquet`

Pour des raison de performances nous nous servirons désormais de fichiers au format parquet. Nous allons commencé par convertir l'ensemble des fichiers `.csv` individuels, puis nous concatènerons les fichiers individuels en un seul fichier `.parquet` comportant toutes les lignes.


#### 2.3.1 Cas du mois d'avril

Le fichier contenant les données du mois d'avril 2016 semble plus corrompu que les autres. Nous lui appliquerons donc un traitement particulier.


In [25]:
ap_data_path = join(raw_data_path, "2016_04.csv")
df_ap_clean = pd.read_csv(ap_data_path, on_bad_lines="warn", low_memory=False)
df_ap_clean.head()

/tmp/ipykernel_151321/127148769.py:2: ParserWarning: Skipping line 121012: expected 65 fields, saw 75
Skipping line 386249: expected 65 fields, saw 83
Skipping line 388291: expected 65 fields, saw 78
Skipping line 389371: expected 65 fields, saw 72
Skipping line 389548: expected 65 fields, saw 81
Skipping line 453858: expected 65 fields, saw 97

  df_ap_clean = pd.read_csv(ap_data_path, on_bad_lines="warn", low_memory=False)


,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,TAIL_NUM,...,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,Unnamed: 64
0,2016,2,4,3,7,2016-04-03,DL,19790,DL,N915DN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,2,4,3,7,2016-04-03,DL,19790,DL,N3755D,...,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,2,4,3,7,2016-04-03,DL,19790,DL,N3755D,...,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,2,4,3,7,2016-04-03,DL,19790,DL,N325US,...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,2,4,3,7,2016-04-03,DL,19790,DL,N366NB,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df_ap_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479950 entries, 0 to 479949
Data columns (total 65 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   YEAR                   479950 non-null  object 
 1   QUARTER                479950 non-null  object 
 2   MONTH                  479950 non-null  int64  
 3   DAY_OF_MONTH           479950 non-null  object 
 4   DAY_OF_WEEK            479950 non-null  object 
 5   FL_DATE                479950 non-null  object 
 6   UNIQUE_CARRIER         479950 non-null  object 
 7   AIRLINE_ID             479950 non-null  int64  
 8   CARRIER                479950 non-null  object 
 9   TAIL_NUM               479185 non-null  object 
 10  FL_NUM                 479950 non-null  object 
 11  ORIGIN_AIRPORT_ID      479950 non-null  object 
 12  ORIGIN_AIRPORT_SEQ_ID  479950 non-null  int64  
 13  ORIGIN_CITY_MARKET_ID  479950 non-null  object 
 14  ORIGIN                 479950 non-nu

Nous constatons rapidement que des colonnes qui devraient être de type int ou float (comme pour le fichier de janvier) sont de type object, ce qui indique probablement un type mixte pour ces colonnes. Analysons quelques unes de ces colonnes problématiques.


In [27]:
cols_to_check = [
    "QUARTER",
    "DAY_OF_MONTH",
    "DAY_OF_WEEK",
    "FL_NUM",
    "CRS_DEP_TIME",
    "CRS_ARR_TIME",
    "ARR_TIME_BLK",
]

for col in cols_to_check:
    print("-" * 50)
    print(col)
    print("-" * 50)
    print(df_ap_clean[col].value_counts())

--------------------------------------------------
QUARTER
--------------------------------------------------
QUARTER
2     362831
1     117118
EV         1
Name: count, dtype: int64
--------------------------------------------------
DAY_OF_MONTH
--------------------------------------------------
DAY_OF_MONTH
25    18088
15    17894
29    17791
22    17762
21    17606
28    17398
27    17249
18    17077
20    17059
24    16783
14    16641
26    16330
6     16285
19    16254
17    15656
3     15608
23    15571
13    15188
8     15041
30    14990
1     14880
12    14874
4     14797
7     14741
11    14618
16    14581
5     14326
2     14002
10    13899
9     12570
31     4390
EV        1
Name: count, dtype: int64
--------------------------------------------------
DAY_OF_WEEK
--------------------------------------------------
DAY_OF_WEEK
5         76515
2         70534
4         69843
3         69261
1         66117
6         64559
7         63120
N707EV        1
Name: count, dtype: int64

Nous constatons certaines valeurs aberrantes parmi les valeurs uniques de certaines colonnes. Une seule ligne semble concernée à chaque fois. Regardons s'il s'agit toujours de la même ligne.


In [28]:
ev_rows = df_ap_clean[df_ap_clean["QUARTER"] == "EV"]
ev_rows

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,TAIL_NUM,...,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,Unnamed: 64
461808,16-03-04,EV,20366,EV,N707EV,5059,10397,1039705,30397,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_ap_clean = df_ap_clean.drop(index=461808)
df_ap_clean[df_ap_clean["QUARTER"] == "EV"]

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,TAIL_NUM,...,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,Unnamed: 64


In [30]:
cols_to_check = [
    "QUARTER",
    "DAY_OF_MONTH",
    "DAY_OF_WEEK",
    "FL_NUM",
    "CRS_DEP_TIME",
    "CRS_ARR_TIME",
    "ARR_TIME_BLK",
]

for col in cols_to_check:
    print("-" * 50)
    print(col)
    print("-" * 50)
    print(df_ap_clean[col].value_counts())

--------------------------------------------------
QUARTER
--------------------------------------------------
QUARTER
2    362831
1    117118
Name: count, dtype: int64
--------------------------------------------------
DAY_OF_MONTH
--------------------------------------------------
DAY_OF_MONTH
25    18088
15    17894
29    17791
22    17762
21    17606
28    17398
27    17249
18    17077
20    17059
24    16783
14    16641
26    16330
6     16285
19    16254
17    15656
3     15608
23    15571
13    15188
8     15041
30    14990
1     14880
12    14874
4     14797
7     14741
11    14618
16    14581
5     14326
2     14002
10    13899
9     12570
31     4390
Name: count, dtype: int64
--------------------------------------------------
DAY_OF_WEEK
--------------------------------------------------
DAY_OF_WEEK
5    76515
2    70534
4    69843
3    69261
1    66117
6    64559
7    63120
Name: count, dtype: int64
--------------------------------------------------
FL_NUM
-------------------

Nous constatons qu'étrangement, ce fichier qui est censé ne concerner que le mois d'avril contient un nombre significatif des valeurs de `QUARTER` à 1 au lieu de 2.

**Hypothèse :** Certaines entrées du mois de mars ont été incluses dans les données du mois d'avril


In [31]:
coherence_check = ((df_ap_clean["MONTH"] == "3") & (df_ap_clean["QUARTER"] == "1")) | (
    (df_ap_clean["MONTH"] == "4") & (df_ap_clean["QUARTER"] == "2")
)

incoherent_rows = df_ap_clean[~coherence_check & df_ap_clean["MONTH"].isin(["3", "4"])]

print(f"Nombre de lignes incohérentes: {len(incoherent_rows)}")
if len(incoherent_rows) > 0:
    print("Échantillon des incohérences:")
    print(incoherent_rows[["MONTH", "QUARTER"]].head())

Nombre de lignes incohérentes: 0


L'hypothèse selon laquelle des lignes du mois de mars sont incluses dans les données du mois d'avril semble se vérifier dans la mesure où il existe une cohérence parfaite entre le mois et le semestre.


In [32]:
jan_float_cols = df_jan.select_dtypes(include=["float64", "float32"]).columns
ap_float_cols = df_ap_clean.select_dtypes(include=["float64", "float32"]).columns
print(len(jan_float_cols))
print(len(ap_float_cols))

30
33


## 4. VISUALISATIONS


## 5.MODÉLISATION
